# GA + ML Trading Optimization Workflow (MACD Edition)

This notebook demonstrates a hybrid approach to trading strategy development using MACD:
1. **Genetic Algorithm (GA)**: Optmizes MACD parameters (Fast, Slow, Signal).
2. **Machine Learning (ML)**: Filters false signals using Random Forest.

---

## 1. Setup & Load Data

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import random
from deap import base, creator, tools, algorithms
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Reload custom modules if edited
%load_ext autoreload
%autoreload 2

from data_loader import load_data
from strategy import MACDStrategy

# Load Logic
df = load_data('EURUSD=X', '2010-01-01', '2024-12-31')
df['Close'].plot(figsize=(12,6), title='EURUSD Price')
plt.show()

## 2. Genetic Algorithm Optimization
We use DEAP to find the best `Fast`, `Slow`, and `Signal` period for MACD.

In [ ]:
# Initialize Strategy Class
strategy = MACDStrategy(df)

# --- GA Setup ---
if hasattr(creator, "FitnessMax"):
    del creator.FitnessMax
if hasattr(creator, "Individual"):
    del creator.Individual

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_fast", random.randint, 5, 50)
toolbox.register("attr_slow", random.randint, 20, 100)
toolbox.register("attr_signal", random.randint, 5, 50)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (toolbox.attr_fast, toolbox.attr_slow, toolbox.attr_signal), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    fast, slow, sig = individual
    if fast >= slow: return -9999,
    return strategy.evaluate(fast, slow, sig),

toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=[5, 20, 5], up=[50, 100, 50], indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run GA
random.seed(42)
pop = toolbox.population(n=50)
result_pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=10, verbose=False)

best_ind = tools.selBest(result_pop, 1)[0]
print(f"Best Params Found: Fast={best_ind[0]}, Slow={best_ind[1]}, Signal={best_ind[2]}")
print(f"Best Return: {best_ind.fitness.values[0]:.4f}")

BEST_FAST = best_ind[0]
BEST_SLOW = best_ind[1]
BEST_SIGNAL = best_ind[2]

## 3. Generate Signals & Features
Using the optimized parameters, we generate signals and extract technical indicators.

In [ ]:
# Generate Signals with Best Params
df_signals = strategy.generate_signals(BEST_FAST, BEST_SLOW, BEST_SIGNAL)

# Add Technical Features for ML
temp_df = df_signals.copy()
temp_df['rsi'] = ta.momentum.rsi(temp_df['Close'], window=14)
temp_df['atr'] = ta.volatility.average_true_range(temp_df['High'], temp_df['Low'], temp_df['Close'])
temp_df['adx'] = ta.trend.adx(temp_df['High'], temp_df['Low'], temp_df['Close'])

# MACD Specific
temp_df['macd_norm'] = temp_df['MACD_Line'] / temp_df['Close']
temp_df['signal_norm'] = temp_df['Signal_Line'] / temp_df['Close']
temp_df['macd_slope'] = temp_df['MACD_Line'].diff()

temp_df.dropna(inplace=True)

# Extract Trade Entry Points Only
temp_df['Prev_Signal'] = temp_df['Signal'].shift(1)
trades_df = temp_df[temp_df['Signal'] != temp_df['Prev_Signal']].copy()

# Labels
def get_trade_result(idx, signal, full_df):
    entry_price = full_df.loc[idx, 'Close']
    future_df = full_df.loc[idx:]
    if len(future_df) < 2: return 0
    
    future_signals = future_df[future_df['Signal'] != signal]
    if len(future_signals) == 0: return 0
    
    exit_price = future_signals.iloc[0]['Close']
    
    if signal == 1: return (exit_price - entry_price) / entry_price
    else: return (entry_price - exit_price) / entry_price

labels = []
returns = []
for idx, row in trades_df.iterrows():
    ret = get_trade_result(idx, row['Signal'], temp_df)
    returns.append(ret)
    labels.append(1 if ret > 0 else 0)

trades_df['Trade_Return'] = returns
trades_df['Label'] = labels

print(f"Total Trades: {len(trades_df)}")
print(f"Profitable: {sum(labels)} ({(sum(labels)/len(labels))*100:.2f}%)")

## 4. Train ML Filter (Random Forest)
Train a classifier to filter out losing trades.

In [ ]:
features = ['rsi', 'atr', 'adx', 'macd_norm', 'signal_norm', 'macd_slope']
X = trades_df[features]
y = trades_df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf.fit(X_train, y_train)

preds = rf.predict(X_test)
print(classification_report(y_test, preds))

importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
importances.plot(kind='bar', title='Feature Importance')
plt.show()